## Hello world of LLM!

In [13]:
from google import genai
from dotenv import load_dotenv
import os

# configuration of llm
from google.genai import types

# Load environment variables from .env file
load_dotenv()
# Ensure the API key is set in the environment
api_key = os.getenv("GOOGLE_GENAI_API_KEY")
system_prompt_default = "You are an assistant that provides concise and accurate answers to user queries. Always say 'Thanks for your question!' at the end of your response."
def call_llm(prompt: str, system_prompt: str = system_prompt_default) -> str:
    """
    Calls the Google GenAI LLM with the provided prompt and returns the response.
    
    Args:
        prompt (str): The input prompt to send to the LLM.
        
    Returns:
        str: The response from the LLM.
    """
    client = genai.Client(api_key=api_key)
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        # config 
        config=types.GenerateContentConfig(
            temperature=0.2,
            max_output_tokens=1000,
            top_p=0.95,
            top_k=40,
            system_instruction=system_prompt, 
        ),
        
    )
    return response.text.strip() if response.text else ""

In [14]:
# Sample prompt
result = call_llm("What is the capital of France?")
print(f"LLM Response: {result}")

LLM Response: The capital of France is Paris.
Thanks for your question!
